In [1]:
import torch
from torch import nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from datetime import timedelta

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class LstmModel(nn.Module):
    def __init__(
        self,
        learning_rate,
        num_layers,
        size,
        size_layer,
        output_size,
        forget_bias=0.1,
    ):
        super(LstmModel, self).__init__()
        self.lstm = nn.LSTM(
            input_size=size,
            hidden_size=size_layer,
            num_layers=num_layers,
            batch_first=True,
            dropout=1 - forget_bias
        )
        self.fc = nn.Linear(size_layer, output_size)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.loss_fn = nn.MSELoss()

    def forward(self, x, hidden):
        output, (hn, cn) = self.lstm(x, hidden)
        return self.fc(output[:, -1, :]), (hn, cn)

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        return (weight.new(self.lstm.num_layers, batch_size, self.lstm.hidden_size).zero_(),
                weight.new(self.lstm.num_layers, batch_size, self.lstm.hidden_size).zero_())

def calculate_accuracy(real, predict):
    real = np.array(real) + 1
    predict = np.array(predict) + 1
    percentage = 1 - np.sqrt(np.mean(np.square((real - predict) / real)))
    return percentage * 100

def anchor(signal, weight):
    buffer = []
    last = signal[0]
    for i in signal:
        smoothed_val = last * weight + (1 - weight) * i
        buffer.append(smoothed_val)
        last = smoothed_val
    return buffer

In [4]:
df = pd.read_csv('../dataset/GOOG-year.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-11-02,778.200012,781.650024,763.450012,768.700012,768.700012,1872400
1,2016-11-03,767.250000,769.950012,759.030029,762.130005,762.130005,1943200
2,2016-11-04,750.659973,770.359985,750.560974,762.020020,762.020020,2134800
3,2016-11-07,774.500000,785.190002,772.549988,782.520020,782.520020,1585100
4,2016-11-08,783.400024,795.632996,780.190002,790.510010,790.510010,1350800


In [5]:
minmax = MinMaxScaler().fit(df.iloc[:, 4:5].astype('float32')) # Close index
df_log = minmax.transform(df.iloc[:, 4:5].astype('float32')) # Close index
df_log = pd.DataFrame(df_log)
df_log.head()

,0
0,0.112708
1,0.090008
2,0.089628
3,0.160459
4,0.188066


In [6]:
test_size = 30
simulation_size = 10

df_train = df_log.iloc[:-test_size]
df_test = df_log.iloc[-test_size:]
df.shape, df_train.shape, df_test.shape

((252, 7), (222, 1), (30, 1))

In [19]:
num_layers = 1
size_layer = 128
timestamp = 5
epoch = 50
dropout_rate = 0.8
future_day = test_size
learning_rate = 0.01

In [8]:
print(df_log.shape[1])

1


In [12]:
def forecast(model, df, df_log, df_train, minmax, learning_rate, num_layers, size_layer, dropout_rate, epoch, timestamp, test_size):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()

    pbar = tqdm(range(epoch), desc='train loop')
    for _ in pbar:
        total_loss, total_acc = [], []
        h_state = model.init_hidden(1)
        
        for k in range(0, df_train.shape[0] - 1, timestamp):
            # 重置梯度
            model.zero_grad()
            
            index = min(k + timestamp, df_train.shape[0] - 1)
            batch_x = torch.FloatTensor(df_train.iloc[k:index, :].values).unsqueeze(0).to(device)
            batch_y = torch.FloatTensor(df_train.iloc[k+1:index+1, :].values).to(device)
            
            # 分离隐藏状态，避免梯度累积
            h_state = (h_state[0].detach(), h_state[1].detach())
            
            logits, h_state = model(batch_x, h_state)
            loss = criterion(logits, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss.append(loss.item())
            total_acc.append(calculate_accuracy(batch_y[:, 0].cpu().numpy(), logits[:, 0].detach().cpu().numpy()))
        
        pbar.set_postfix(cost=np.mean(total_loss), acc=np.mean(total_acc))

    future_day = test_size
    output_predict = np.zeros((df_train.shape[0] + future_day, df_train.shape[1]))
    output_predict[0] = df_train.iloc[0]
    upper_b = (df_train.shape[0] // timestamp) * timestamp

    model.eval()
    with torch.no_grad():
        h_state = model.init_hidden(1)
        for k in range(0, upper_b, timestamp):
            x = torch.FloatTensor(df_train.iloc[k:k+timestamp].values).unsqueeze(0).to(device)
            out, h_state = model(x, h_state)
            output_predict[k+1:k+timestamp+1] = out.cpu().numpy()

        if upper_b != df_train.shape[0]:
            x = torch.FloatTensor(df_train.iloc[upper_b:].values).unsqueeze(0).to(device)
            out, h_state = model(x, h_state)
            output_predict[upper_b+1:df_train.shape[0]+1] = out.cpu().numpy()
            future_day -= 1
            date_ori.append(date_ori[-1] + timedelta(days=1))

        for i in range(future_day):
            x = torch.FloatTensor(output_predict[-future_day-timestamp+i:-future_day+i]).unsqueeze(0).to(device)
            out, h_state = model(x, h_state)
            output_predict[-future_day+i] = out[-1].cpu().numpy()
            date_ori.append(date_ori[-1] + timedelta(days=1))

    output_predict = minmax.inverse_transform(output_predict)
    deep_future = anchor(output_predict[:, 0], 0.3)
    
    return deep_future[-test_size:]

In [13]:
def run_simulations(model_class, df, df_log, df_train, minmax, learning_rate, num_layers, size_layer, dropout_rate, epoch, timestamp, test_size, simulation_size):
    results = []
    for i in range(simulation_size):
        print(f'simulation {i + 1}')
        # 为每次模拟创建一个新的模型实例
        model = model_class(
            learning_rate=learning_rate,
            num_layers=num_layers,
            size=df_log.shape[1],
            size_layer=size_layer,
            output_size=df_log.shape[1],
            forget_bias=dropout_rate
        )
        result = forecast(model, df, df_log, df_train, minmax, learning_rate, num_layers, size_layer, dropout_rate, epoch, timestamp, test_size)
        results.append(result)
    return results

In [17]:
simulation_results = run_simulations(LstmModel, df, df_log, df_train, minmax, learning_rate, num_layers, size_layer, dropout_rate, epoch, timestamp, test_size, simulation_size)

c:\Users\hades\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.19999999999999996 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


simulation 1


train loop:   0%|          | 0/50 [00:00<?, ?it/s]c:\Users\hades\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
train loop: 100%|██████████| 50/50 [00:03<00:00, 15.90it/s, acc=97.4, cost=0.00158] 
c:\Users\hades\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.19999999999999996 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


simulation 2


train loop:   0%|          | 0/50 [00:00<?, ?it/s]c:\Users\hades\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
train loop: 100%|██████████| 50/50 [00:03<00:00, 14.72it/s, acc=97, cost=0.00201]   
c:\Users\hades\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.19999999999999996 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


...

train loop:   0%|          | 0/50 [00:00<?, ?it/s]c:\Users\hades\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
train loop: 100%|██████████| 50/50 [00:03<00:00, 13.03it/s, acc=98, cost=0.000892]  
c:\Users\hades\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.19999999999999996 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


simulation 8


train loop:   0%|          | 0/50 [00:00<?, ?it/s]c:\Users\hades\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
train loop: 100%|██████████| 50/50 [00:03<00:00, 14.01it/s, acc=97.3, cost=0.00214] 
c:\Users\hades\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.19999999999999996 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


simulation 9


train loop:   0%|          | 0/50 [00:00<?, ?it/s]c:\Users\hades\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
train loop:  40%|████      | 20/50 [00:01<00:02, 14.15it/s, acc=97.6, cost=0.00165]